-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Propagating Incremental Updates with Structured Streaming and Delta Lake

## Learning Objectives
By the end of this lab, you should be able to:
* **Apply your knowledge of structured streaming and Auto Loader to implement a simple multi-hop architecture**

## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook. Note that re-executing this cell will allow you to start the lab over.

In [0]:
%run ../Includes/Classroom-Setup-7.2L

Python interpreter will be restarted.
Python interpreter will be restarted.

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_7_2l"

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/7.2l
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/7.2l/7_2l.db
 DA.paths.checkpoints: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/7.2l/_checkpoints

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_7_2l:
 -none-

Setup completed in 2 seconds

## Ingest data(Creating bronze table)

This lab uses a collection of customer-related CSV data from DBFS found in */databricks-datasets/retail-org/customers/*.

##### Read this data using Auto Loader using its schema inference (use **`customers_checkpoint_path`** to store the schema info). Stream the raw data to a Delta table called **`bronze`**.

In [0]:
# ANSWER
customers_checkpoint_path = f"{DA.paths.checkpoints}/customers"

query = (spark.readStream
              .format("cloudFiles")
              .option("cloudFiles.format", "csv")
              .option("cloudFiles.schemaLocation", customers_checkpoint_path)
              .load("/databricks-datasets/retail-org/customers/")
              .writeStream
              .format("delta")
              .option("checkpointLocation", customers_checkpoint_path)
              .outputMode("append")
              .table("bronze"))

In [0]:
DA.block_until_stream_is_ready(query)

The stream has processed 2 batchs

Run the cell below to check your work.

In [0]:
assert spark.table("bronze"), "Table named `bronze` does not exist"
assert spark.sql(f"SHOW TABLES").filter(f"tableName == 'bronze'").first()["isTemporary"] == False, "Table is temporary"
assert spark.table("bronze").dtypes ==  [('customer_id', 'string'), ('tax_id', 'string'), ('tax_code', 'string'), ('customer_name', 'string'), ('state', 'string'), ('city', 'string'), ('postcode', 'string'), ('street', 'string'), ('number', 'string'), ('unit', 'string'), ('region', 'string'), ('district', 'string'), ('lon', 'string'), ('lat', 'string'), ('ship_to_address', 'string'), ('valid_from', 'string'), ('valid_to', 'string'), ('units_purchased', 'string'), ('loyalty_segment', 'string'), ('_rescued_data', 'string')], "Incorrect Schema"

Let's create a streaming temporary view into the bronze table, so that we can perform transforms using SQL.

In [0]:
(spark
  .readStream
  .table("bronze")
  .createOrReplaceTempView("bronze_temp"))

## Clean and enhance data

Using CTAS syntax, define a new streaming view called **`bronze_enhanced_temp`** that does the following:
* Skips records with a null **`postcode`** (set to zero)
* Inserts a column called **`receipt_time`** containing a current timestamp
* Inserts a column called **`source_file`** containing the input filename

In [0]:
%sql
-- TODO
CREATE OR REPLACE TEMPORARY VIEW bronze_enhanced_temp AS
SELECT *, current_timestamp() receipt_time, input_file_name() source_file
from bronze_temp 
where postcode > 0
  

Run the cell below to check your work.

In [0]:
assert spark.table("bronze_enhanced_temp"), "Table named `bronze_enhanced_temp` does not exist"
assert spark.sql(f"SHOW TABLES").filter(f"tableName == 'bronze_enhanced_temp'").first()["isTemporary"] == True, "Table is not temporary"
assert spark.table("bronze_enhanced_temp").dtypes ==  [('customer_id', 'string'), ('tax_id', 'string'), ('tax_code', 'string'), ('customer_name', 'string'), ('state', 'string'), ('city', 'string'), ('postcode', 'string'), ('street', 'string'), ('number', 'string'), ('unit', 'string'), ('region', 'string'), ('district', 'string'), ('lon', 'string'), ('lat', 'string'), ('ship_to_address', 'string'), ('valid_from', 'string'), ('valid_to', 'string'), ('units_purchased', 'string'), ('loyalty_segment', 'string'), ('_rescued_data', 'string'), ('receipt_time', 'timestamp'), ('source_file', 'string')], "Incorrect Schema"
assert spark.table("bronze_enhanced_temp").isStreaming, "Not a streaming table"

# Creating Silver table

Stream the data from **`bronze_enhanced_temp`** to a table called **`silver`**.

In [0]:
# TODO
silver_checkpoint_path = f"{DA.paths.checkpoints}/silver"

query = (spark.table("bronze_enhanced_temp")
  .writeStream
  .format('delta')
  .option("checkpointLocation", silver_checkpoint_path)
  .outputMode("append")
  .table("silver"))

In [0]:
DA.block_until_stream_is_ready(query)

The stream has processed 2 batchs

Run the cell below to check your work.

In [0]:
assert spark.table("silver"), "Table named `silver` does not exist"
assert spark.sql(f"SHOW TABLES").filter(f"tableName == 'silver'").first()["isTemporary"] == False, "Table is temporary"
assert spark.table("silver").dtypes ==  [('customer_id', 'string'), ('tax_id', 'string'), ('tax_code', 'string'), ('customer_name', 'string'), ('state', 'string'), ('city', 'string'), ('postcode', 'string'), ('street', 'string'), ('number', 'string'), ('unit', 'string'), ('region', 'string'), ('district', 'string'), ('lon', 'string'), ('lat', 'string'), ('ship_to_address', 'string'), ('valid_from', 'string'), ('valid_to', 'string'), ('units_purchased', 'string'), ('loyalty_segment', 'string'), ('_rescued_data', 'string'), ('receipt_time', 'timestamp'), ('source_file', 'string')], "Incorrect Schema"
assert spark.table("silver").filter("postcode <= 0").count() == 0, "Null postcodes present"

Let's create a streaming temporary view into the silver table, so that we can perform business-level using SQL.

In [0]:
(spark
  .readStream
  .table("silver")
  .createOrReplaceTempView("silver_temp"))

## Creating Gold tables

Using CTAS syntax, define a new streaming view called **`customer_count_temp`** that counts customers per state.

In [0]:
%sql
-- TODO
CREATE OR REPLACE TEMPORARY VIEW customer_count_temp AS
SELECT state ,count(customer_id) as customer_count 
from silver_temp
group by
state

Run the cell below to check your work.

In [0]:
assert spark.table("customer_count_temp"), "Table named `customer_count_temp` does not exist"
assert spark.sql(f"SHOW TABLES").filter(f"tableName == 'customer_count_temp'").first()["isTemporary"] == True, "Table is not temporary"
assert spark.table("customer_count_temp").dtypes ==  [('state', 'string'), ('customer_count', 'bigint')], "Incorrect Schema"

Finally, stream the data from the **`customer_count_temp`** view to a Delta table called **`gold_customer_count_by_state`**.

In [0]:
# ANSWER
customers_count_checkpoint_path = f"{DA.paths.checkpoints}/customers_counts"

query = (spark.table("customer_count_temp")
              .writeStream
              .format("delta")
              .option("checkpointLocation", customers_count_checkpoint_path)
              .outputMode("complete")
              .table("gold_customer_count_by_state"))

In [0]:
DA.block_until_stream_is_ready(query)

The stream has processed 2 batchs

Run the cell below to check your work.

In [0]:
assert spark.table("gold_customer_count_by_state"), "Table named `gold_customer_count_by_state` does not exist"
assert spark.sql(f"show tables").filter(f"tableName == 'gold_customer_count_by_state'").first()["isTemporary"] == False, "Table is temporary"
assert spark.table("gold_customer_count_by_state").dtypes ==  [('state', 'string'), ('customer_count', 'bigint')], "Incorrect Schema"
assert spark.table("gold_customer_count_by_state").count() == 51, "Incorrect number of rows" 

## Query the results

Query the **`gold_customer_count_by_state`** table (this will not be a streaming query). Plot the results as a bar graph and also using the map plot.

In [0]:
%sql
SELECT * FROM gold_customer_count_by_state
order by customer_count desc
limit 5

state,customer_count
NY,3363
CA,2874
FL,2517
OH,1914
MA,1889


## Wrapping Up

Run the following cell to remove the database and all data associated with this lab.

In [0]:
DA.cleanup()

Stopping the stream "None"
Stopping the stream "None"
Stopping the stream "None"
Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_7_2l"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/7.2l"

By completing this lab, you should now feel comfortable:
* **Using PySpark to configure Auto Loader for incremental data ingestion**
* **Using Spark SQL to aggregate streaming data**
* **Streaming data to a Delta table**

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>